In [2]:
from FinChatbot.utils.common import call_data
import pickle
import os

In [3]:
tables, texts = call_data(file_path = "../Data/example_pdf/2023q4-alphabet-earnings-release.pdf")

In [5]:
len(tables), len(texts)

(15, 51)

In [25]:
def save_data(data, file_name, type):
    os.makedirs("../artifacts", exist_ok = True)
    if type == "summary" or type == "original":
        os.makedirs(f"../artifacts/{str(type)}", exist_ok = True)
        file_path = f"../artifacts/{str(type)}/{str(file_name)}.pkl"
        with open(file_path, "wb") as file:
            pickle.dump(data, file)
        print(f"Data saved succesfully at: {file_path}")

In [26]:
save_data(data = tables,
          file_name = "table_original",
          type = "original")

save_data(data = texts,
          file_name = "text_original",
          type = "original")

Data saved succesfully at: ../artifacts/original/table_original.pkl
Data saved succesfully at: ../artifacts/original/text_original.pkl


In [7]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [8]:
import os

In [9]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [11]:
prompt_text = '''   
Generate a concise and accurate financial summary of a company's performance in a single paragraph based on the provided data, the data can be either table or a text. \
The summary should report precise values and trends directly from the data, emphasizing critical metrics such as revenue, profit, growth trends, significant changes, and anomalies. \
It must use clear, professional language without jargon, focusing on key aspects like profitability, cash flow, debt levels, and performance ratios. Assumptions or estimates should not be included. \
The paragraph should also highlight trends or insights relevant for predictive or comparative queries, such as growth expectations, decline patterns, or industry benchmarks.

Data: {element}
'''

In [12]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [13]:
model = ChatOpenAI(temperature=0, model = "gpt-4o-mini")
summary_chain = {"element": lambda x : x} | prompt | model | StrOutputParser()

In [14]:
table_summaries = []
table_summaries = summary_chain.batch(tables, {'max_concurrency':5})

In [15]:
table_summaries

['For the year ended December 31, 2023, the company reported revenues of $307,394, reflecting a 9% increase from $282,836 in 2022, with a notable 13% growth in the fourth quarter compared to the previous year. Operating income rose to $84,293, up from $74,842, maintaining a consistent operating margin of 27%. Net income also improved significantly, reaching $73,795, a 23% increase from $59,972 in 2022, while diluted earnings per share increased from $4.56 to $5.80. Additionally, the company reversed prior losses in other income, reporting a net gain of $1,424 in 2023 compared to a loss of $3,514 in 2022. Overall, the financial performance indicates strong growth trends and improved profitability, positioning the company favorably for future periods.',
 'For the quarter ended December 31, 2023, the company reported total revenues of $86,310 million, reflecting a robust increase of 13.5% from $76,048 million in the same quarter of 2022. Google Services, which includes Google Search and Y

In [16]:
text_summaries = []
text_summaries = summary_chain.batch(texts, {'max_concurrency':5})

In [17]:
text_summaries[:5]

["In the fourth quarter and fiscal year ended December 31, 2023, Alphabet Inc. reported a revenue of $76.5 billion, reflecting a year-over-year increase of 10% from $69.5 billion in 2022. The company's net income for the quarter was $18.2 billion, up from $15.1 billion in the same period last year, resulting in a profit margin of 23.8%. For the full year, Alphabet achieved a net income of $70.5 billion, compared to $59.0 billion in 2022, indicating a robust growth trend in profitability. Operating cash flow remained strong at $25 billion for the quarter, contributing to a healthy cash position. The company's total debt levels were stable at $14 billion, maintaining a low debt-to-equity ratio, which supports its financial resilience. Overall, Alphabet's performance demonstrates solid growth expectations and a strong competitive position within the technology sector.",
 'In the latest performance update, Sundar Pichai, CEO, highlighted the robust growth in the Search segment, alongside i

In [22]:
save_data(data = tables,
          file_name = "table_summaries",
          type = "summary")

save_data(data = texts,
          file_name = "text_summaries",
          type = "summary")

Data saved succesfully at: ../artifacts/summaries/table_summaries.pkl
Data saved succesfully at: ../artifacts/summaries/text_summaries.pkl
